# TinyVLM

## Data Exploration and Initial Preprocessing

### Data Exploration

##### Data

- [Images used for training with descriptions](https://huggingface.co/datasets/BAAI/CapsFusion-120M)

General Info on this Data:

- This dataset provides over 13 million image links, but we are scaling down. We downloaded the first 5 million rows of the dataset, and of these we will only use the rows where the image link gives a successful response code. For the purpose of data exploration, we are just using the first 100,000 rows.
- All initial images are not uniform in any regard, however during preprocessing, all images will be cropped (either center cropped or padded)
- 3 different descriptions for each images as features

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv('data/image_metadata_0.csv')
df.head()

,image_url,capsfusion,identifier,original_width,original_height
0,http://ih3.redbubble.net/image.12080909.2547/f...,"The Lego minifigure, known as Minifig [Rainbow...",515623f9-0f34-49b1-be2c-253636badaf6,NaN,NaN
1,https://cdn.shopify.com/s/files/1/2161/7557/pr...,"The Chilly Grip H2O Waterproof Thermal Lined, ...",ccb9ec94-d0d4-4384-b728-a8d9ad150aa6,439.0,480.0
2,http://i0.wp.com/www.ladycarehealth.com/wp-con...,Abdominal cramping is a common sign of pregnan...,642a0186-bbc4-4244-b87b-617d4ea350ed,NaN,NaN
3,https://cdn.shopify.com/s/files/1/2986/1514/pr...,This lovely 1930s Velveteen Burgundy Half Slee...,9be0e45a-bdd0-410d-b825-77b3781eaa84,800.0,1156.0
4,https://cdn.shopify.com/s/files/1/2169/9777/pr...,The Bubblegum Divas Store offers a Little Yell...,b75f96ac-c759-415c-91de-4502da4a3156,100.0,100.0


In [4]:
print(f'Shape of our Dataframe{df.shape}')
print()

Shape of our Dataframe(100000, 5)



We are also using the following datasets for instruction tuning. The first dataset below contains questions, and second contains answers.

- [Instruction Tuning: VQA](https://visualqa.org/download.html)


In [ ]:
df_q.head()

,image_id,question,question_id
0,458752,What is this photo taken looking through?,458752000
1,458752,What position is this man playing?,458752001
2,458752,What color is the players shirt?,458752002
3,458752,Is this man a professional baseball player?,458752003
4,262146,What color is the snow?,262146000


In [ ]:
df_a.head()

,question_type,multiple_choice_answer,answers,image_id,answer_type,question_id
0,what is this,net,"[{'answer': 'net', 'answer_confidence': 'maybe...",458752,other,458752000
1,what,pitcher,"[{'answer': 'pitcher', 'answer_confidence': 'y...",458752,other,458752001
2,what color is the,orange,"[{'answer': 'orange', 'answer_confidence': 'ye...",458752,other,458752002
3,is this,yes,"[{'answer': 'yes', 'answer_confidence': 'yes',...",458752,yes/no,458752003
4,what color is the,white,"[{'answer': 'white', 'answer_confidence': 'yes...",262146,other,262146000


### Preprocessing Steps

For preprocessing, we plan on doing the following:

- Downloading only images in which gives a successful response code (rows in the dataset corresponding to images without a successful response code will be disregarded)
- Cropping all the images to a desired 128 x 128 dimension
- Normalization is likely not needed, however will perform when needed
- Classification of the data, classifying each of the features
- Encorporate image descriptions to the desired images for training
- Prepare questions and answers for images to do instruction Tuning to the LLM pre-train model

Our dataset consists of images with a wide variety of aspect ratios. Some images are already square or nearly square, whereas others have extreme aspect ratios (very narrow/wide). To account for this, we will set an aspect ratio threshold of 0.6, where aspect ratio is defined as the minimum of the width and height over the maximum of the width and height. For images with an aspect ratio greater than or equal to 0.6, we will center crop the image, and images with an aspect ratio less than 0.6 will be padded. An example of our preprocessing for a single image is as follows: Say we have a very narrow image with a height of 400px and a width of 100px. The image will be padded to make it square, meaning black bars will be added to the left and right of the image, each one having a height of 400px and a width of 150px. The image will then be downscaled to 128x128.

Our function for preprocessing an image is below:

In [10]:
def preprocess_image(self, img):
    """
    Adaptively choose preprocessing method based on aspect ratio
    """
    if img.mode != 'RGB':
        img = img.convert('RGB')
    
    width, height = img.size
    aspect_ratio = min(width, height) / max(width, height)
    
    # Track which method was used
    if aspect_ratio >= self.aspect_ratio_threshold:
        # Use center crop for images with good aspect ratio
        for transform in self.crop_transforms:
            img = transform(img)
        with self.stats_lock:
            self.stats['crop_count'] += 1
    else:
        # Use padding for images with extreme aspect ratios
        img = self.resize_and_pad(img)
        with self.stats_lock:
            self.stats['pad_count'] += 1
    
    return img

We will also use CLIP labels in order to encode our images to certain text labels from our CLIP labels that were created. In the process of making the CLIP labels, we had a few example results with the encoded result from it. From here we would pass in these samples into GPT in order to help generate more samples given a certain categorical inputs. From here, we are able to expand our CLIP labels from 10 for each individual categories we had, to 50-100 new labels for each different categories. Here, we will be able to pass this in for our pretraining to encode our images for classification.

### Data preparing for instruction tuning

1. Download questions and answers from the dataset
2. Create DataFrame from json data files which include questions and answers
3. Combine questions and answers and create a new DataFrame according to the image_id and answer_id
4. Create answers into a complete sentences
5. Add tags to indicate system prompt, user questions, and answers from the model
6. Combine system prompt, user questions, and answers into one col and label them with corrsponding image_id
7. Output the data as csv file